In [2]:
%load_ext autoreload
%autoreload 2
import os
import os.path
import sklearn.model_selection
import sklearn.linear_model
import sklearn.ensemble
import spacy
import sys
from sklearn.feature_extraction.text import CountVectorizer
from anchor import anchor_text
from myUtils import *
import pickle
import matplotlib.pyplot as plt

In [3]:
# dataset from http://www.cs.cornell.edu/people/pabo/movie-review-data/
# Link: http://www.cs.cornell.edu/people/pabo/movie-review-data/rt-polaritydata.tar.gz
def load_polarity(path='sentiment-sentences'):
    data = []
    labels = []
    f_names = ['rt-polarity.neg', 'rt-polarity.pos']
    for (l, f) in enumerate(f_names):
        for line in open(os.path.join(path, f), 'rb'):
            try:
                line.decode('utf8')
            except:
                continue
            data.append(line.strip())
            labels.append(l)
    return data, labels

Note: you must have spacy installed. Run:

        pip install spacy && python -m spacy download en_core_web_sm

If you want to run BERT, you have to install transformers and torch or tf: 

        pip install torch transformers spacy && python -m spacy download en_core_web_sm
        

In [4]:
nlp = spacy.load('en_core_web_sm')

In [5]:
data, labels = load_polarity()
train, test, train_labels, test_labels = sklearn.model_selection.train_test_split(data, labels, test_size=.2, random_state=42)
train, val, train_labels, val_labels = sklearn.model_selection.train_test_split(train, train_labels, test_size=.1, random_state=42)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)
val_labels = np.array(val_labels)
counter_test, counter_test_labels = TextUtils.counter_test()

In [6]:
vectorizer = CountVectorizer(min_df=1)
vectorizer.fit(train)
train_vectors = vectorizer.transform(train)
test_vectors = vectorizer.transform(test)
val_vectors = vectorizer.transform(val)
counter_test_vectors = vectorizer.transform(counter_test)

In [7]:
c = sklearn.linear_model.LogisticRegression()
# c = sklearn.ensemble.RandomForestClassifier(n_estimators=500, n_jobs=10)
c.fit(train_vectors, train_labels)
preds = c.predict(val_vectors)
print('Val accuracy', sklearn.metrics.accuracy_score(val_labels, preds))
def predict_lr(texts):
    return c.predict(vectorizer.transform(texts))

Val accuracy 0.7544910179640718


/home/almr16/miniconda3/envs/cs236781-hw/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Using BERT

In [9]:
explainer = anchor_text.AnchorText(nlp, ['negative', 'positive'], use_unk_distribution=False)

RuntimeError: CUDA out of memory. Tried to allocate 86.00 MiB (GPU 0; 10.76 GiB total capacity; 195.11 MiB already allocated; 18.44 MiB free; 208.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
test = [example.decode('utf-8') for example in test]
anchor_examples = [example.decode('utf-8') for example in train]
anchor_examples = [example for example in anchor_examples if 20< len(example) < 70 and len(example)>20][:500]

In [9]:
pickle.dump( test, open( "results/text_test.pickle", "wb" ))
pickle.dump( test_labels, open( "results/text_test_labels.pickle", "wb" ))

In [ ]:
my_utils = TextUtils(anchor_examples, counter_test, explainer, predict_lr)
#explanations = my_utils.compute_explanations(np.random.choice(len(test), exps_num))
explanations = my_utils.compute_explanations(list(range(0, len(anchor_examples))))

In [ ]:
pickle.dump( explanations, open( "results/text_exps.pickle", "wb" ))

# Loading Results

In [48]:
explanations = pickle.load( open( "results/text_exps.pickle", "rb" ))
test = pickle.load( open( "results/text_test.pickle", "rb" ))
test = np.array(test)
test_labels = pickle.load( open( "results/text_test_labels.pickle", "rb" ))

In [49]:
myUtils =  TextUtils(counter_test, counter_test, None, None, "results/test_exps.pickle")
explanations = myUtils.remove_duplicates(explanations)

In [66]:
explanations = [ExtendedExplanation(exp, anchor_examples, counter_test, counter_test_labels, predict_lr, explainer) for exp in explanations if len(exp.fit_examples) >0]

In [67]:
#explanations = [exp for exp in explanations if len(exp.fit_examples) > 10] 
explanations.sort(key=lambda exp: exp.test_precision)

In [68]:
best = explanations[-10:]
best.reverse()

for exp in best:
    print("------------------------")
    exp_label =  predict_lr([anchor_examples[exp.index]])[0]
    print('Prediction:', explainer.class_names[exp_label])
    print('Anchor: %s' % (' AND '.join(exp.names)))
    print('Precision: %.2f' % exp.precision)
    print('Coverage: %.2f' % exp.coverage)
    print('Anchor test precision: %.2f' % exp.test_precision)
    print('Anchor test coverage: %.2f' % (exp.test_cov))
    covered_labels = counter_test_labels[exp.fit_examples]
    real_percentage = np.mean(covered_labels == exp_label)
    print('Anchor test REAL precision: %.2f' % exp.real_precision)

------------------------
Prediction: negative
Anchor: pointless
Precision: 1.00
Coverage: 0.00
Anchor test precision: 1.00
Anchor test coverage: 0.01
Anchor test REAL precision: 0.73
------------------------
Prediction: negative
Anchor: unfunny
Precision: 1.00
Coverage: 0.00
Anchor test precision: 1.00
Anchor test coverage: 0.01
Anchor test REAL precision: 1.00
------------------------
Prediction: negative
Anchor: incoherent
Precision: 0.99
Coverage: 0.00
Anchor test precision: 1.00
Anchor test coverage: 0.01
Anchor test REAL precision: 0.83
------------------------
Prediction: negative
Anchor: wannabe
Precision: 0.96
Coverage: 0.00
Anchor test precision: 1.00
Anchor test coverage: 0.01
Anchor test REAL precision: 0.60
------------------------
Prediction: negative
Anchor: lacking
Precision: 0.96
Coverage: 0.00
Anchor test precision: 1.00
Anchor test coverage: 0.00
Anchor test REAL precision: 1.00
------------------------
Prediction: negative
Anchor: secret AND too
Precision: 1.00
Cover

In [69]:
len(explanations)

207

In [80]:
exps = explanations
exps = [exp for exp in exps if len(exp.fit_examples)>2]
real_precisions = [exp.real_precision for exp in exps]
test_precisions = [exp.test_precision for exp in exps]

In [81]:
plt.scatter(test_precisions, real_precisions, s = range(len(exps)), alpha = 0.9)
plt.xlabel('predicted class precision')
plt.ylabel('real class precision')
plt.title('logistic regression')
plt.savefig("results/text.png")

In [82]:
img = plt.imread("results/text.png")
plt.figure(figsize = (5,5))
plt.axis('off')
_ = plt.imshow(img)